In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('dataset/adult.data', engine='python', delimiter=', ',encoding='latin1', header=None)
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'class']  
df.drop(labels=['fnlwgt', 'education-num'], axis=1, inplace=True)
df.head()

In [32]:
mappings = {}
mappings['workclass'] = {'Private':0, 'Self-emp-not-inc':1, 'Self-emp-inc':2, 'Federal-gov':3, 'Local-gov':4, 'State-gov':5, 'Without-pay':6, 'Never-worked':7, '?':8}
mappings['education'] = {'Preschool':0, '1st-4th':1, '5th-6th':2, '7th-8th': 3, '9th':4, '10th':5, '11th':6, '12th':7,'HS-grad':8, 'Prof-school':9, 'Assoc-acdm':10, 'Assoc-voc':11,'Some-college':12, 'Bachelors':13,  'Masters':14, 'Doctorate':15 }
mappings['marital-status'] = {'Married-civ-spouse':0, 'Divorced':1, 'Never-married':2, 'Separated':3, 'Widowed':4, 'Married-spouse-absent':5, 'Married-AF-spouse':6}
mappings['occupation'] = {'Tech-support':0, 'Craft-repair':1, 'Other-service':2, 'Sales':3, 'Exec-managerial':4, 'Prof-specialty':5, 'Handlers-cleaners':6, 'Machine-op-inspct':7, 'Adm-clerical':8, 'Farming-fishing':9, 'Transport-moving':10, 'Priv-house-serv':11, 'Protective-serv':12, 'Armed-Forces':13}
mappings['relationship'] = {'Not-in-family':0, 'Husband':1, 'Wife':2, 'Own-child':3, 'Unmarried':4, 'Other-relative':5}
mappings['race'] = {'White':0, 'Black':1, 'Asian-Pac-Islander':2, 'Amer-Indian-Eskimo':3, 'Other':4}
mappings['sex'] = {'Male':0, 'Female':1}
mappings['native-country'] = {'United-States':0, 'Cambodia':1, 'England':2, 'Puerto-Rico':3, 'Canada':4, 'Germany':5, 'Outlying-US(Guam-USVI-etc)':6, 'India':7, 'Japan':8, 'Greece':9, 'South':10, 'China':11, 'Cuba':12, 'Iran':13, 'Honduras':14, 'Philippines':15, 'Italy':16, 'Poland':17, 'Jamaica':18, 'Vietnam':19, 'Mexico':20, 'Portugal':21, 'Ireland':22, 'France':23, 'Dominican-Republic':24, 'Laos':25, 'Ecuador':26, 'Taiwan':27, 'Haiti':28, 'Columbia':29, 'Hungary':30, 'Guatemala':31, 'Nicaragua':32, 'Scotland':33, 'Thailand':34, 'Yugoslavia':35, 'El-Salvador':36, 'Trinadad&Tobago':37, 'Peru':38, 'Hong':39, 'Holand-Netherlands':40}
mappings['class'] = {'<=50K':0, '>50K':1}
for k in mappings:
    df[k] = df[k].map(mappings[k])
df.head()

In [32]:
if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import train_test_split
else:
    from sklearn.model_selection import train_test_split

y=df.as_matrix(columns=[df.columns[0]])
X=df.as_matrix(columns=[df.columns[1:]])
X_train, X_test, y_train, y_test =\ train_test_split(X, y, test_size=0.3, random_state=0)


In [32]:
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)
print(X_train_std)

[(51, 'Mean'), (22, 'Median'), (14, 'Mode'), (4, 'MLTV'), (4, 'DL.1'), (4, 'ASTV'), (3, 'e'), (3, 'b'), (3, 'Width'), (3, 'Variance'), (3, 'UC.1'), (3, 'SUSP'), (3, 'Nzeros'), (3, 'Nmax'), (3, 'Min'), (3, 'Max'), (3, 'MSTV'), (3, 'LD'), (3, 'E'), (3, 'DS.1'), (3, 'DS'), (3, 'DR'), (3, 'DP.1'), (3, 'DP'), (3, 'DL'), (3, 'DE'), (3, 'CLASS'), (3, 'C'), (3, 'AD'), (3, 'AC.1'), (3, 'AC'), (2, 'UC'), (2, 'Tendency'), (2, 'NSP'), (2, 'FS'), (2, 'FM.1'), (2, 'FM'), (2, 'D'), (2, 'ALTV'), (2, 'A'), (1, 'B')]
['Mean', 'Median']


In [48]:
d = {most_important[i] : data.get(most_important[i]) for i in range(len(most_important))}
x = pd.DataFrame(data=d)

In [58]:
from sklearn.decomposition import PCA
plotX = most_important[0]
plotY = most_important[1]
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = [plotX, plotY])
principalDf

,Mean,Median
0,9.842862,-14.174142
1,-2.316502,0.459717
2,-0.224769,-0.330631
3,1.188341,-0.386493
4,-0.959255,-1.009255
5,41.378178,-4.098235
6,42.056801,-4.832722
7,19.502906,-2.525814
8,19.502906,-2.525814
9,19.502906,-2.525814


In [60]:
sp = principalDf.plot.scatter(x=plotX,y=plotY,c="Red")
warnings.filterwarnings('ignore')

sp.set_xlabel(plotX, fontsize = 15)
sp.set_ylabel(plotY, fontsize = 15)
sp.set_title('2 component PCA', fontsize = 20)

fig = plt.figure(figsize= (x[plotX].max(),x[plotY].max()))

plt.show()
warnings.filterwarnings('always')

In [28]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
